In [30]:
import pandas as pd

df = pd.read_csv('cust_act1.csv')

len(set(df["browser"]))

13

In [ ]:
import plotly.express as px

fig = px.histogram(df, "browser", 
                   facet_col="region", 
                   color="region",
                   title="Counts of browser per region",
                   labels={"browser": "browser"},
                   height=1000, 
                   facet_col_wrap=2, 
                   facet_col_spacing=0.1,
                   )
fig.update_layout(showlegend=False, xaxis={'categoryorder':'total descending'})
fig.update_yaxes(matches=None, showticklabels=True)
# fig.update_xaxes(showticklabels=True)
fig.show()

In [ ]:
import numpy as np

def percentnull(df_column):
    print(f"{np.round(100 * df_column.isna().sum()/len(df_column), 2)}% of values in the <{df_column.name}> column are null.")

percentnull(df["product_related"])

4.7% of values in the <product_related> column are null.


# Chart Boilerplate Collection

#### 1. Transparent Circle Scatter Plot

In [ ]:
# This is copied straight from another db analysis.
# TODO all variables and categories must be replaced with relevant ones.

category_array = ["I have never written code", "< 1 years", "1-2 years", "3-5 years", "5-10 years", "10-20 years", "20+ years"]
# fig = px.scatter(programming_experience_salary_df, "Programming Experience", "Salary", title="Density of Programming Experience vs Salary")
fig = px.scatter(programming_experience_salary_df, "Programming Experience", "Salary", facet_col=df["Q2"],title="Density of Programming Experience vs Salary")
fig.update_traces(marker=dict(
            opacity=0.05,
            size=20,
            line=dict(
                color='MediumPurple',
                width=0.5
            )))
fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':category_array})
fig.show()